In [1]:
from __future__ import division
import glob
import numpy
import pickle
from pyspark import SparkContext, AccumulatorParam, StorageLevel
import random

sc = SparkContext( 'spark://headnodehost:7077', 'pyspark')

In [2]:
#this is the test matrix size I have been using - generate random data of this size
matsize = (4000,127,24)
rand_tmp = numpy.random.normal(size=matsize)

#parallelize the data structures (in the actual application I keep track of NaN values, not negatives)
pdata_tmp = sc.parallelize(rand_tmp, numSlices=12)
lt0_tmp = sc.parallelize(rand_tmp < 0, numSlices=12)

#zip the two data structures together so that maps will iterate over them in parallel
pdata_test = pdata_tmp.zip(lt0_tmp)

#persist and count the data structure, print out size and contents of first 2-element list
print(pdata_test.persist().count())
print(pdata_test.first()[0].shape, pdata_test.first()[1].shape)
print(pdata_test.first())

4000
((127L, 24L), (127L, 24L))
(array([[-0.7225378 , -0.59842533, -0.71624953, ...,  0.61728871,
        -0.24987318,  0.72968711],
       [-0.66455189,  0.41094483, -0.55515486, ...,  0.33986751,
        -0.57250438,  0.29709322],
       [-0.53917298, -0.99933304,  0.30988386, ...,  0.03454377,
        -0.6294595 , -1.69648396],
       ..., 
       [ 0.23895443,  2.10604574,  0.26785619, ...,  0.18028419,
         0.75845765, -0.12320117],
       [-0.31825316,  0.48587111, -0.8662242 , ...,  0.26701833,
         0.62417484, -0.17163352],
       [ 0.61557427, -1.72388158,  0.39542576, ..., -1.47422382,
         0.14549535,  1.01414954]]), array([[ True,  True,  True, ..., False,  True, False],
       [ True, False,  True, ..., False,  True, False],
       [ True,  True, False, ..., False,  True,  True],
       ..., 
       [False, False, False, ..., False, False,  True],
       [ True, False,  True, ..., False, False,  True],
       [False,  True, False, ...,  True, False, False]], dt

In [6]:
#the next step in the data prep is to subtract the global mean, which in this example is 0.001
gmean = sc.broadcast(0.001)
test = pdata_test.map(lambda x: [x[0] - gmean.value, x[1]])
test.persist().count()
#test.count()

4000

In [7]:
#this should print out a data structure that is the same size and shape as pdata_test.first(), but instead I always get a 
#socket write error if test is persisted. Strangely, this call to test.first() works just fine if test is not persisted...
print(test.first())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 4 times, most recent failure: Lost task 0.3 in stage 8.0 (TID 55, workernode0.sparktesttim2.c10.internal.cloudapp.net): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:113)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:159)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:121)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:409)
	at org.apache.spark.api.p